In [1]:
from dlgo.data.parallel_processor import GoDataProcessor
from dlgo.encoders.oneplane import OnePlaneEncoder
from dlgo.networks.small import SMALL

import torch
import torch.nn as nn
from torch.optim import SGD

In [2]:
def compute_acc(argmax, y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i] == y[i]:
            count += 1
    return count / len(argmax)

In [4]:
go_board_rows, go_board_cols = 19, 19
num_classes = go_board_rows * go_board_cols
num_games = 100
BATCH_SIZE = 128
LEARNING_RATE = 0.001
NUM_EPOCHES = 5

In [5]:
encoder = OnePlaneEncoder((go_board_rows, go_board_cols))  # First we create an encoder of board size.

processor = GoDataProcessor(encoder=encoder.name())  # Then we initialize a Go Data processor with it.

generator = processor.load_go_data('train', num_games, use_generator=True)  # From the processor we create two data generators, for training and testing.
test_generator = processor.load_go_data('test', num_games, use_generator=True)

model = SMALL(go_board_rows, encoder.num_planes).cuda()

optimizer = SGD(model.parameters())
loss_fn = nn.CrossEntropyLoss()
model.train()

>>> Downloading index page
KGS-2019_04-19-1255-.tar.gz 1255
KGS-2019_03-19-1478-.tar.gz 1478
KGS-2019_02-19-1412-.tar.gz 1412
KGS-2019_01-19-2095-.tar.gz 2095
KGS-2018_12-19-1992-.tar.gz 1992
KGS-2018_11-19-1879-.tar.gz 1879
KGS-2018_10-19-1209-.tar.gz 1209
KGS-2018_09-19-1587-.tar.gz 1587
KGS-2018_08-19-1447-.tar.gz 1447
KGS-2018_07-19-949-.tar.gz 949
KGS-2018_06-19-1002-.tar.gz 1002
KGS-2018_05-19-1590-.tar.gz 1590
KGS-2018_04-19-1612-.tar.gz 1612
KGS-2018_03-19-833-.tar.gz 833
KGS-2018_02-19-1167-.tar.gz 1167
KGS-2018_01-19-1526-.tar.gz 1526
KGS-2017_12-19-1488-.tar.gz 1488
KGS-2017_11-19-945-.tar.gz 945
KGS-2017_10-19-1351-.tar.gz 1351
KGS-2017_09-19-1353-.tar.gz 1353
KGS-2017_08-19-2205-.tar.gz 2205
KGS-2017_07-19-1191-.tar.gz 1191
KGS-2017_06-19-910-.tar.gz 910
KGS-2017_05-19-847-.tar.gz 847
KGS-2017_04-19-913-.tar.gz 913
KGS-2017_03-19-717-.tar.gz 717
KGS-2017_02-19-525-.tar.gz 525
KGS-2017_01-19-733-.tar.gz 733
KGS-2016_12-19-1208-.tar.gz 1208
KGS-2016_11-19-980-.tar.gz 980
KGS

SMALL(
  (layer1): Sequential(
    (0): ZeroPad2d((3, 3, 3, 3))
    (1): Conv2d(1, 48, kernel_size=(7, 7), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): ZeroPad2d((2, 2, 2, 2))
    (1): Conv2d(48, 32, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer3): Sequential(
    (0): ZeroPad2d((2, 2, 2, 2))
    (1): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): ZeroPad2d((2, 2, 2, 2))
    (1): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer5): Sequential(
    (0): Linear(in_features=11552, out_features=512, bias=True)
    (1): ReLU(inplace=True)
  )
  (layer6): Sequential(
    (0): Linear(in_features=512, out_features=361, bias=True)
    (1): Softmax(dim=-1)
  )
)

In [6]:
for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x, y in generator.generate(BATCH_SIZE, num_classes):
        optimizer.zero_grad()
        x = x.cuda()
        y_ = model(x)
        loss = loss_fn(y_, y.cuda()) 
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1, tot_loss))
    if epoch % 2 == 1:
        x, y = next(iter(test_generator))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_, dim=-1)
        test_acc = compute_acc(argmax, y)

        print("Acc(val) : {}".format(test_acc))

c:\Anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not NoneType

In [14]:
model

SMALL(
  (layer1): Sequential(
    (0): ZeroPad2d((3, 3, 3, 3))
    (1): Conv2d(1, 48, kernel_size=(7, 7), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): ZeroPad2d((2, 2, 2, 2))
    (1): Conv2d(48, 32, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer3): Sequential(
    (0): ZeroPad2d((2, 2, 2, 2))
    (1): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): ZeroPad2d((2, 2, 2, 2))
    (1): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (2): ReLU(inplace=True)
  )
  (layer5): Sequential(
    (0): Linear(in_features=11552, out_features=512, bias=True)
    (1): ReLU(inplace=True)
  )
  (layer6): Sequential(
    (0): Linear(in_features=512, out_features=361, bias=True)
    (1): Softmax(dim=-1)
  )
)

In [9]:
y

tensor([263, 317, 297, 316, 295, 296, 277, 315, 276, 302, 250, 249, 230, 268,
        231, 116, 308, 289, 309, 326, 110,  70,  49, 108,  52,  71,  34,  53,
         33,  35,  74,  55,  73,  54, 130, 147, 167, 166, 186, 173,  40,  59,
         41,  42,  23,  24,  43,  61,  25,  22,   5,  21,  88, 109,  62,  81,
         82, 100, 101, 119, 174, 154, 283, 282, 264, 262, 243, 242, 320, 301,
        261, 281, 224,  50,  31,  30,  29,  32,  11,  51, 333, 314, 313, 334,
        290, 270, 248, 267, 192, 332, 331, 352, 293, 191, 210, 193, 194, 211,
        212, 192, 175, 233, 214, 253, 216, 255, 237, 232, 213, 120, 121,  87,
        107,  68,  69,  48,  30, 125, 126, 145, 106, 105, 124,  47,  28,  46,
        104,  86], device='cuda:0', dtype=torch.int32)